## Parametri

#### lista di elementi bx, bz dimensione delle travi, px e py dimensione dei pilastri e collezioni dy1,dy2.. hz1,hz2 rispettivamente le distane fra pilastri e l'alteza dell'interpiani. Parametri che con rispettivi cicli for vengono acquisiti e istanziati. Oltre ciò, è stata definita una funzione ggplBoneStrutture che prende in input un file csv di vettori , e output invece di un hpc verranno istanziati i rispettivi parametri. Il compito della "costruzione" dello scheletro della struttura in cemento armato sarà implementata in un'altra funzione. I modelli presi in considerazione saranno: 

![img](modello1.jpeg)
![img](modello2.gif)

In [ ]:
from pyplasm import *
import csv

bx = by = .6	
bz = .2
px = py = .2			
dy = 3.4
dy2 = 4.5		
hz = 2.8
hz1 = 2.0

dim_beam = (bx,bz)
dim_pillar=(px,py)
dis_pillars = [dy,dy2]
heights_interstory = [hz,hz1]

def ggpl_bone_structure(file_name):
	with open(file_name,'rb'):
		reader = csv.reader(f)
		for r in reader:
			print r

# Struttura principale e i rispettivi componenti

#### funzione che preso i parametri istanziati precedentemente crea la prima struttura. La base dove poggiano i pilastri è grande rispettivamente 3volte la grandezza del pilastro(cosicchè i pilastri verranno posizionati al centro esato della base), grazie al prodotto cartesiano operazioni quindi di QUOTE E PROD verranno istaziate 6 basi nell'asse delle x e 4 basi nell'asse delle y con altezza 0.2 e il rispettivo prodotto cartesiano. Tale oggetto hpc verrà chiamato base.
#### Stesso discorso vale anche per i pilastri, in più verranno distanziati (ogni pilastro) con la dimensione dy parametrica, acquisita precedentemente, e la rispettiva alteza data da hz.
#### Le travi le ho suddivise in: travi asse x e travi asse y. Queste sono state create nello stesso modo visto precedentemente, l'unica differenza chiaramente è nel "posizionamento e misure".
#### La funione di ritorno inece di un singolo oggetto hpc è una lista con gli elementi precedenti, scelta di progetto più elastica per risolvere in futuro problemi che richiedono solo una parte della struttura intera.

In [ ]:

def struttura(dim_beam,dim_pillar,dis_pillars,heights_interstory):
	
	x_base = QUOTE([px*3, - dis_pillars[0]] * 6)   
	y_base = QUOTE([py*3, - dis_pillars[0]] * 4)  

	base = INSR(PROD)([x_base,y_base,Q(.2)]) 
	
	x_pilastri = QUOTE([-dim_pillar[0], dim_pillar[0], -dim_pillar[0], - dis_pillars[0] ] * 6)	
	y_pilastri = QUOTE([-dim_pillar[0], dim_pillar[0], -dim_pillar[0], - dis_pillars[0] ] * 4)	
	pilastri = INSR(PROD)([x_pilastri,y_pilastri,QUOTE([-.2,heights_interstory[0]]*5 )])   
	
	x_travi_assey = QUOTE([dim_beam[0], -(dis_pillars[0])] *6)
	y_travi_assey = QUOTE([4*(dis_pillars[0]) -dim_beam[0]])
	travi_assey = INSR(PROD)([x_travi_assey,y_travi_assey,QUOTE([-dim_beam[1] ,.2 - heights_interstory[0], dim_beam[1]] * 5)])

	x_travi_assex = QUOTE([dis_pillars[0]*6])
	y_travi_assex = QUOTE([dim_beam[0],-dis_pillars[0]] * 4)
	travi_assex = INSR(PROD)([x_travi_assex,y_travi_assex,QUOTE([-dim_beam[1] ,.2 - heights_interstory[0], dim_beam[1]] * 5)])
	
	return (base,pilastri,travi_assey,travi_assex)

# Stutture secondarie

#### Per rispecchiare il modello vedi immagine all'inizio, è stata scelta la "costruzione" di altre 2 strutture secondarie. Sono state create nello stesso modo della struttura 1, la sola differenza è che hanno una dimensione più piccola, ma più alta, rispetto alla struttura principale.  Per il posizionamento è stata usata la funzione T (traslate) di una quantità adatta a non sovrapporsi con la struttura principale, ma neanche distaccarsi da rendere non realistica la rappresentazione. 

In [ ]:

x_base_seconda = QUOTE([px*3, - dis_pillars[1]] * 2)   
b_seconda = INSR(PROD)([x_base_seconda,x_base_seconda,Q(.2)])
base_seconda = STRUCT([T(1)(3.8*dy),T(2)(3.8*dy),b_seconda])
base_terza = STRUCT([T(1)(3.85*dy),T(2)(-dy*1.65),b_seconda])

x_pilastri_seconda = QUOTE([-dim_pillar[0], dim_pillar[0], -dim_pillar[1], - dis_pillars[1] ] * 2)
p_seconda = INSR(PROD)([x_pilastri_seconda,x_pilastri_seconda,QUOTE([-.2,heights_interstory[0]]*7 )])
pilastri_seconda = STRUCT([T(1)(3.8*dy),T(2)(3.8*dy),p_seconda])
pilastri_terza = STRUCT([T(1)(3.9*dy),T(2)(-1.65*dy),p_seconda])

x_traviy_seconda = QUOTE([dim_beam[0], -(dis_pillars[1])] *2)
y_traviy_seconda = QUOTE([1.5*(dis_pillars[1]) -dim_beam[0]])
t_assey_seconda = INSR(PROD)([x_traviy_seconda,y_traviy_seconda,QUOTE([-dim_beam[1] ,.2 - heights_interstory[0], dim_beam[1]] * 7)])
travi_assey_seconda = STRUCT([T(1)(3.8*dy),T(2)(3.8*dy),t_assey_seconda])
travi_assey_terza = STRUCT([T(1)(3.85*dy),T(2)(-1.65*dy),t_assey_seconda])

x_travi_seconda = QUOTE([dis_pillars[1]*1.35])
y_travi_seconda = QUOTE([dim_beam[0],-dis_pillars[1]] * 2)
t_assex_seconda =  INSR(PROD)([x_travi_seconda,y_travi_seconda,QUOTE([-dim_beam[1] ,.2 - heights_interstory[0], dim_beam[1]] * 7)])
travi_assex_seconda = STRUCT([T(1)(3.8*dy),T(2)(3.8*dy),t_assex_seconda])
travi_assex_terza = STRUCT([T(1)(3.8*dy),T(2)(-1.65*dy),t_assex_seconda])

## Test e View Scheletro del sistema della Struttura in Cemento armato

#### sono stati affrontati test, principalmente sui calcoli matematici affrontati, per non incorrere in errori di progettazione e parallelamente le visualizzazioni delle componenti dello scheletro del sistema di struttura in cemento armato.


![img](base.png)
![img](pilastri.png)
![img](monoTravi.png)
![img](travi.png)

In [ ]:

test_base = struttura(dim_beam,dim_pillar,dis_pillars,heights_interstory)
VIEW(STRUCT([test_base[0],base_seconda,base_terza]))
VIEW(STRUCT([test_base[0],test_base[1],base_seconda,base_terza,pilastri_terza,pilastri_seconda]))
VIEW(STRUCT([test_base[0],test_base[1],test_base[2],base_seconda,base_terza,pilastri_terza,pilastri_seconda,travi_assey_seconda,travi_assey_terza]))
VIEW(STRUCT([test_base[0],test_base[1],test_base[2],test_base[3],base_seconda,base_terza,pilastri_terza,pilastri_seconda,travi_assey_seconda,travi_assey_terza,travi_assex_seconda,travi_assex_terza]))

# Struttura finale

#### In questa fase è stato ragruppato tutto in un unico hpc ed è stata data una forma onsistente alle travi della struttura.

In [ ]:
x_travi = QUOTE([bx,(bx*6) + ( dy*5)])
y_travi = QUOTE([by,(by*4) + ( dy*3) - 0.2])
travi = INSR(PROD)([x_travi,y_travi,QUOTE([bz ,.2 - hz, bz] * 5)])

x_travi_seconda = QUOTE([bx,1.5*dy])
y_travi_seconda = QUOTE([by,1.5*dy])
t_seconda = INSR(PROD)([x_travi_seconda,y_travi_seconda,QUOTE([bz ,.2 - hz, bz] * 7)])
travi_seconda = STRUCT([T(1)(3.8*dy),T(2)(3.8*dy),t_seconda])

travi_terza = STRUCT([T(1)(3.85*dy),T(2)(-1.65*dy),t_seconda])

VIEW(STRUCT([test_base[0],test_base[1],travi,base_seconda,base_terza,pilastri_seconda,pilastri_terza,travi_terza,travi_seconda]))

![img](struttura.png)